In [3]:
import pandas as pd
import numpy as np
pd.options.mode.copy_on_write = True

In [4]:
path_prefix_fitted = "../../data/IEDB/fitted_data"
path_prefix_cleaned = "../../data/IEDB/cleaned_data"

missing_column_names = ['TRAC', 'TRBC', 'TRA_leader', 'TRB_leader', 'Linker', 'Link_order', 
                'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq', 'TRB_3_prime_seq', 'score']

needed_column_names = ['TRAC', 'TRBC', 'TRA_leader', 'TRB_leader', 'Linker', 'Link_order', #nan columns
                'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq', 'TRB_3_prime_seq', #nan columns
                'Epitope', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3', 'TCR_name',
                'MHC A', 'MHC B', 'MHC class'] 

df_cleaned_beta = pd.read_csv(f"{path_prefix_fitted}/IEDB_beta_fitted.csv")
df_cleaned_paired = pd.read_csv(f"{path_prefix_fitted}/IEDB_paired_fitted.csv")

# Assign missing columns with default value np.nan
df_beta = df_cleaned_beta.assign(**{col: np.nan for col in missing_column_names})
df_paired = df_cleaned_paired.assign(**{col: np.nan for col in missing_column_names})

# Remove rows with nan values
#df_beta.dropna(subset=["Epitope", "TRBV", "TRBJ", "TRB_CDR3"], inplace=True)
df_beta.dropna(subset=["Epitope", "TRB_CDR3"], inplace=True)
#df_paired.dropna(subset=["Epitope", "TRAV", "TRAJ", "TRA_CDR3", "TRBV", "TRBJ", "TRB_CDR3"], inplace=True)
df_paired.dropna(subset=["Epitope", "TRA_CDR3", "TRB_CDR3"], inplace=True)

df_beta.to_csv(f"{path_prefix_cleaned}/IEDB_cleaned_data_beta.csv", index=False, columns=needed_column_names)
df_paired.to_csv(f"{path_prefix_cleaned}/IEDB_cleaned_data_paired.csv", index=False, columns=needed_column_names)

/tmp/ipykernel_9836/1317125257.py:12: DtypeWarning: Columns (3,4,5,6,7,8,9,13,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned_beta = pd.read_csv(f"{path_prefix_fitted}/IEDB_beta_fitted.csv")
